In [ ]:
import sys
import os

parent = os.path.dirname(os.path.realpath("../../"))
sys.path.append(parent)

import numpy as np
import scipy
import tqdm
import open3d as o3d
import matplotlib.pyplot as plt
import glob


from core import *
from utils import phantom_builder
from utils import geometry

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Make Phantom

In [2]:
voxel_size = np.array([0.001, 0.001, 0.001])
# voxel_size = np.array([0.0005, 0.0005, 0.0005])
surface_mesh = o3d.io.read_triangle_mesh(
    f"{parent}/assets/cardiac_TEE_phantom/esophageal_surface.ply"
)
body_mask = phantom_builder.voxelize(voxel_size[0], mesh=surface_mesh)

In [ ]:
test_phantom = phantom.Phantom(
    voxel_dims=(voxel_size[0], voxel_size[0], voxel_size[0]),
    matrix_dims=body_mask.shape,
    baseline=(1540, 1000),
    seed=None,
)

blood = tissue.Tissue(name="blood", c=1578, rho=1060, sigma=1.3, scale=0.00001, label=1)
myocardium = tissue.Tissue(
    name="myocardium", c=1592, rho=1081, sigma=20, scale=0.0001, label=2
)
esophagus = tissue.Tissue(
    name="esophagus", c=1500, rho=1100, sigma=10, scale=0.0001, label=3
)
fat = tissue.Tissue(name="fat", c=1480, rho=970, sigma=15, scale=0.0001, label=4)

heart_file_dir = f"{parent}/assets/cardiac_TEE_phantom/"
heart_tissue_list = [blood, myocardium, esophagus]

test_phantom.build_organ_from_mesh(
    surface_mesh, voxel_size[0], heart_tissue_list, dir_path=heart_file_dir
)
test_phantom.set_default_tissue(fat)

In [4]:
test = test_phantom.get_complete()

In [ ]:
index = 50
plt.imshow(test[0, :, :, index], cmap="gray", vmin=1450, vmax=1650)
plt.imshow(
    body_mask[:, :, index], alpha=(body_mask[:, :, index] > 0) * 0.9, cmap="Reds"
)

# Set up simulation

In [6]:
num_transducers = 6
transducers = [
    transducer.Focused(
        max_frequency=2e6,
        elements=128,
        width=20e-3,
        height=20e-3,
        sensor_sampling_scheme="not_centroid",
        sweep=np.pi / 2,
        ray_num=128,
        imaging_ndims=2,
        focus_azimuth=100e-3,
        focus_elevation=150e-3,
        cycles=3,
    )
    for i in range(num_transducers)
]

for t in transducers:
    t.make_sensor_coords(test_phantom.baseline[0])

test_transducer_set = transducer_set.TransducerSet(transducers, seed=8888)

In [7]:
test_transducer_set = transducer_set.TransducerSet(transducers, seed=8888)

# apical 4 chamber
transform1 = geometry.Transform([-np.pi * 0.75, 0, 0], [0, 0, 0])
transform2 = geometry.Transform([0, -np.pi / 2 * 0.1, 0], [0, 0, 0])
transform3 = geometry.Transform([0, 0, 0], [0, 0, 0])
transform = transform1 * transform2 * transform3
position = [-0.15, -0.01, -0.05]
transform.translation = (transform1 * transform2).apply_to_point(
    position, inverse=False
)
test_transducer_set.assign_pose(0, transform)

# apical 2 chamber
transform1 = geometry.Transform([-np.pi * 0.75, 0, 0], [0, 0, 0])
transform2 = geometry.Transform([0, -np.pi / 2 * 0.1, 0], [0, 0, 0])
transform3 = geometry.Transform([0, 0, -np.pi / 2], [0, 0, 0])
transform = transform1 * transform2 * transform3
position = [-0.15, -0.01, -0.05]
transform.translation = (transform1 * transform2).apply_to_point(
    position, inverse=False
)
test_transducer_set.assign_pose(1, transform)

# parasternal shout axis papillary
transform1 = geometry.Transform([-np.pi * 0.8, 0, 0], [0, 0, 0])
transform2 = geometry.Transform([0, np.pi / 2 * 0.7, 0], [0, 0, 0])
transform3 = geometry.Transform([-np.pi * 0.15, 0, -np.pi / 2 * 0.3], [0, 0, 0])
transform = transform1 * transform2 * transform3
position = np.array([0.095, 0.04, 0.04])
transform.translation = position
test_transducer_set.assign_pose(2, transform)

# parasternal short axis mitral
transform1 = geometry.Transform([-3.01825409, 0.87793401, -0.87338513], [0, 0, 0])
transform2 = geometry.Transform([0, 0, 0], [0, 0, 0])
transform3 = geometry.Transform([0, 0, 0], [0, 0, 0])
transform = transform1 * transform2 * transform3
position = np.array([0.09, 0.03, 0.056])
transform.translation = position
test_transducer_set.assign_pose(3, transform)

# parasternal long axis
transform1 = geometry.Transform([-3.14, 0.95, -2.51], [0, 0, 0])
transform2 = geometry.Transform([-0.2, 0, 0], [0, 0, 0])
transform3 = geometry.Transform([0, 0.04, 0], [0, 0, 0])
transform = transform1 * transform2 * transform3
position = np.array([0.09, 0.025, 0.045])
transform.translation = position
test_transducer_set.assign_pose(4, transform)

# subcostal 4 chamber
transform1 = geometry.Transform([3, -0.75, 0], [0, 0, 0])
transform2 = geometry.Transform([0, 0, 0], [0, 0, 0])
transform3 = geometry.Transform([0, 0, 0], [0, 0, 0])
transform = transform1 * transform2 * transform3
position = np.array([0.10, 0, -0.15])
transform.translation = position
test_transducer_set.assign_pose(5, transform)


# # parasternal short axis aortic
# # suprasternal aortic arch
# print(transform.rotation.as_euler('ZYX', degrees=False))

In [ ]:
# points = np.array((o3d.io.read_triangle_mesh(f"{parent}/assets/cardiac_TEE_phantom/02_myocardium.obj")).sample_points_uniformly(5000).points)
points = np.array(
    (o3d.io.read_triangle_mesh(f"{parent}/assets/cardiac_TEE_phantom/01_bloodpool.obj"))
    .sample_points_uniformly(8000)
    .points
)

test_transducer_set.plot_transducer_coords(
    scale=0.1, phantom_coords=points, view=(90, 0)
)
test_transducer_set.plot_transducer_coords(
    scale=0.1, phantom_coords=points, view=(0, 0)
)
test_transducer_set.plot_transducer_coords(
    scale=0.1, phantom_coords=points, view=(0, 90)
)

In [9]:
test_sensor = sensor.Sensor(
    transducer_set=test_transducer_set, aperture_type="transmit_as_receive"
)

In [10]:
simprops = simulation.SimProperties(
    grid_size=(200e-3, 25e-3, 25e-3),
    voxel_size=(0.5e-3, 0.5e-3, 0.5e-3),
    PML_size=(16, 16, 16),
    PML_alpha=2,
    t_end=12e-5,  # [s]
    bona=6,  # parameter b/a determining degree of nonlinear acoustic effects
    alpha_coeff=0.5,  # [dB/(MHz^y cm)]
    alpha_power=1.5,
)

In [ ]:
test_experiment = experiment.Experiment(
    simulation_path=f"cardiac_TTE_newphantom",
    sim_properties=simprops,
    phantom=test_phantom,
    transducer_set=test_transducer_set,
    sensor=test_sensor,
    nodes=1,
    results=None,
    indices=None,
    workers=4,
    additional_keys=[],
)

test_experiment.save()

In [12]:
sensor_centroids = []
for i in range(len(test_transducer_set)):
    sensor_centroids.append(
        np.mean(
            test_sensor.sensor_coords[
                i * test_sensor.sensor_coords.shape[0] // len(test_transducer_set) : (
                    i + 1
                )
                * test_sensor.sensor_coords.shape[0]
                // len(test_transducer_set),
                :,
            ],
            axis=0,
        )
        / voxel_size
        + np.array(test.shape[1:]) / 2
    )

In [13]:
# test_experiment.visualize_sensor_mask(index=[slice(0, -1, 1), slice(0, -1, 1), int(np.round(sensor_centroids[0][2]))])
# test_experiment.plot_ray_path(64)

In [14]:
# test_experiment.visualize_sensor_mask(index=[slice(0, -1, 1), slice(0, -1, 1), int(np.round(sensor_centroids[1][2]))])
# test_experiment.plot_ray_path(128+64)

In [15]:
# test_experiment.visualize_sensor_mask(index=[slice(0, -1, 1), slice(0, -1, 1), int(np.round(sensor_centroids[2][2]))])
# test_experiment.plot_ray_path(256+64)

In [16]:
# test_experiment.visualize_sensor_mask(index=[slice(0, -1, 1), slice(0, -1, 1), int(np.round(sensor_centroids[3][2]))])
# test_experiment.plot_ray_path(384+64)

In [17]:
# test_experiment.visualize_sensor_mask(index=[slice(0, -1, 1), slice(0, -1, 1), int(np.round(sensor_centroids[4][2]))])
# test_experiment.plot_ray_path(512+64)

In [18]:
# test_experiment.visualize_sensor_mask(index=[slice(0, -1, 1), slice(0, -1, 1), int(np.round(sensor_centroids[5][2]))])
# test_experiment.plot_ray_path(640+64)

# Run Simulation

In [ ]:
test_experiment = experiment.Experiment.load(f"cardiac_TTE_newphantom")
test_experiment.run(dry=True)

In [ ]:
test_experiment.run(repeat=False)

# Reconstruction

In [21]:
test_experiment.add_results()

In [22]:
test_reconstruction = reconstruction.DAS(experiment=test_experiment)

In [ ]:
signals = test_reconstruction.get_signals(
    dimensions=2, matsize=512, downsample=1, workers=16, tgc=10
)

In [ ]:
plt.imshow(signals[0][:, ::-1], vmin=0, vmax=100000, cmap="gray")
plt.gca().set_facecolor("black")
plt.gca().set_xticks([])
plt.gca().set_yticks([])
plt.savefig("cardiac_TTE_ap4ch_2MHz.png", bbox_inches="tight", pad_inches=0, dpi=300)

plt.imshow(signals[1][:, ::-1], vmin=0, vmax=100000, cmap="gray")
plt.gca().set_facecolor("black")
plt.gca().set_xticks([])
plt.gca().set_yticks([])
plt.savefig("cardiac_TTE_ap2ch_2MHz.png", bbox_inches="tight", pad_inches=0, dpi=300)

plt.imshow(signals[2][:, ::-1], vmin=0, vmax=100000, cmap="gray")
plt.gca().set_facecolor("black")
plt.gca().set_xticks([])
plt.gca().set_yticks([])
plt.savefig(
    "cardiac_TTE_parasternal_sax_midpap_2MHz.png",
    bbox_inches="tight",
    pad_inches=0,
    dpi=300,
)

plt.imshow(signals[3][:, ::-1], vmin=0, vmax=100000, cmap="gray")
plt.gca().set_facecolor("black")
plt.gca().set_xticks([])
plt.gca().set_yticks([])
plt.savefig(
    "cardiac_TTE_parasternal_sax_mitral_2MHz.png",
    bbox_inches="tight",
    pad_inches=0,
    dpi=300,
)

plt.imshow(signals[4][:, ::-1], vmin=0, vmax=100000, cmap="gray")
plt.gca().set_facecolor("black")
plt.gca().set_xticks([])
plt.gca().set_yticks([])
plt.savefig(
    "cardiac_TTE_parasternal_lax_2MHz.png", bbox_inches="tight", pad_inches=0, dpi=300
)

plt.imshow(signals[5][:, ::-1], vmin=0, vmax=100000, cmap="gray")
plt.gca().set_facecolor("black")
plt.gca().set_xticks([])
plt.gca().set_yticks([])
plt.savefig(
    "cardiac_TTE_subcostal_4ch_2MHz.png", bbox_inches="tight", pad_inches=0, dpi=300
)